In [1]:
#---import libraries relating to data manipulation, visualization---------------------------------------------------
import pandas as pd
%matplotlib inline
import numpy as np
import json


#-------import libraries to evaluate the deep learning model---------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer


import warnings
warnings.filterwarnings('ignore')

#------ import Deep Learning Libraries-------#

import torch
import torch.nn as nn
import torch.optim as optim

import optuna
from optuna.trial import Trial

import pickle


pd.set_option('display.max_columns', None)
pd.set_option("max_colwidth", None)

import sys
sys.path.append('../../../src')  # Add the 'src' directory to the path

#### Import Data Loading and Processing Functions
from multimodal.helper_functions import   reproducible_comp
from multimodal.model_architecture import Generic1DCNN
reproducible_comp()

In [2]:
file_path = "../../../data/processed_data/press_in_df.pkl"
with open(file_path, 'rb') as picklefile:
    data = pickle.load(picklefile)

In [3]:
labels_raw = data.iloc[:,0].copy()
features=data.iloc[:,1].values.reshape(-1,1).copy()
print("Features Shape: ", features.shape)
print("Labels Shape: ", labels_raw.shape)

Features Shape:  (360, 1)
Labels Shape:  (360,)


In [4]:
# Create a dictionary for the mapping
mapping = {
    'W': 'Worn',
    'WP': 'Worn-Position',
    'R': 'Referenz',
    'D': 'Dünn',
    'S': 'Slug',
    'SW': 'Slug-Worn',
    'SD': 'Slug-Dünn',
    'WD': 'Worn-Dünn',
    'SP': 'Slug-Position',
    'SDP': 'Slug-Dünn-Position',
    'WDP': 'Worn-Dünn-Position',
    'SWP': 'Slug-Worn-Position',
    'RV': 'Referenz-Offset',
    'WV': 'Worn-Offset',
    'SV': 'Slug-Offset',
    'SDV': 'Slug-Dünn-Offset',
    'WDV': 'Worn-Dünn-Offset',
    'SWV': 'Slug-Worn-Offset'
}


labels_six = labels_raw.map(mapping)

In [5]:
labels_four_mlb = labels_six.str.split('-')
# Use MultiLabelBinarizer to create binary columns for multilabel classification
mlb = MultiLabelBinarizer()
labels_four_mlb = mlb.fit_transform(labels_four_mlb)
category_names = list(mlb.classes_)

In [6]:
labels = labels_four_mlb.copy()
features = features.copy()
print("Features Shape: ", features.shape)
print("Labels Shape: ", labels.shape)

Features Shape:  (360, 1)
Labels Shape:  (360, 6)


In [7]:
max_length = 350
feat = np.array([np.pad(np.array(record), (0, max_length - len(record)), mode='constant', constant_values=(0)) for row in features for record in row])


print("Updated Features Shape: ", feat.shape)

Updated Features Shape:  (360, 350)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(feat, labels, test_size=0.1, 
                                                    random_state=2, shuffle=True, stratify=labels)

In [9]:
X_train = X_train.reshape(-1, 1, feat.shape[1])
X_test = X_test.reshape(-1, 1, feat.shape[1])

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(324, 1, 350)
(324, 6)
(36, 1, 350)
(36, 6)


In [10]:
from torch.utils.data import DataLoader, TensorDataset

# Convert your data to PyTorch tensors
x_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)

input_channels = 1
num_classes = 6

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the objective function for hyperparameter optimization
def objective(trial: Trial):
    try:
        # Define hyperparameters to search for
        lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)  # Suggest a learning rate in log scale
        dropout_prob = trial.suggest_float("dropout_prob", 0.0, 0.5)
        hidden_channels = trial.suggest_int("hidden_channels", 16, 256)
        n_layers = trial.suggest_int("n_layers", 1, 7)
        kernel_size = trial.suggest_int("kernel_size", 3, 9)
        stride = trial.suggest_int("stride", 1, 7)
        padding = trial.suggest_int("padding", 1, 10)
        pooling_type = trial.suggest_categorical("pooling_type", ["max", "avg"])  
        use_batch_norm = trial.suggest_categorical("use_batch_norm", [True, False]) 


        # Split the data into training and validation sets
        x_train_split, x_valid_split, y_train_split, y_valid_split = train_test_split(
            x_train, y_train, test_size=0.2, random_state=42
        )

        train_dataset = TensorDataset(x_train_split, y_train_split)
        valid_dataset = TensorDataset(x_valid_split, y_valid_split)

        train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

        # Create the model with the suggested hyperparameters
        model = Generic1DCNN(input_channels, num_classes, n_layers, kernel_size, stride, padding,
                            hidden_channels, dropout_prob, pooling_type, use_batch_norm).to(device)

        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        num_epochs = 50

        model.train()
        for epoch in range(num_epochs):
            # Training loop
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                outputs = model(inputs)
                loss = criterion(outputs, labels.float())
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

            train_loss = running_loss / len(train_loader)

            # Validation loss calculation (similar to training loop)
            model.eval()
            valid_loss = 0.0
            with torch.no_grad():
                for inputs_valid, labels_valid in valid_loader:
                    inputs_valid, labels_valid = inputs_valid.to(device), labels_valid.to(device)
                    outputs_valid = model(inputs_valid)
                    loss_valid = criterion(outputs_valid, labels_valid.float())
                    valid_loss += loss_valid.item()

            valid_loss = valid_loss / len(valid_loader)
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss}, Validation Loss: {valid_loss}")

        # Calculate and return the final validation loss
        return valid_loss
    
    except Exception as e:
        # Print the error for debugging purposes (optional)
        print(f"Trial failed with error: {e}")

        # Return a high validation loss to indicate failure
        return 1.0  # High validation loss
    


In [11]:
# Create an Optuna study with SQLite storage and TPE pruner
study = optuna.create_study(
    study_name="Case#11: Force Press In: Updated Clasees",
    direction="minimize",  # Minimize final validation loss
    storage="sqlite:///../../database_sqllite//db.sqlite3",
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5),  
)

# Define the number of trials (hyperparameter combinations) you want to search
num_trials = 100  

# Perform hyperparameter optimization
study.optimize(objective, n_trials=num_trials, n_jobs=1)


[I 2024-06-04 15:11:41,917] A new study created in RDB with name: Case#11: Force Press In: Updated Clasees


Epoch 10/50, Training Loss: 0.6773626327514648, Validation Loss: 0.6674076318740845
Epoch 20/50, Training Loss: 0.6667564868927002, Validation Loss: 0.6545446515083313
Epoch 30/50, Training Loss: 0.656006121635437, Validation Loss: 0.6429637372493744
Epoch 40/50, Training Loss: 0.6600508689880371, Validation Loss: 0.6327367722988129


[I 2024-06-04 15:11:43,891] Trial 0 finished with value: 0.623038113117218 and parameters: {'lr': 1.0613856699497563e-05, 'dropout_prob': 0.014830572678247889, 'hidden_channels': 112, 'n_layers': 2, 'kernel_size': 5, 'stride': 4, 'padding': 7, 'pooling_type': 'avg', 'use_batch_norm': False}. Best is trial 0 with value: 0.623038113117218.


Epoch 50/50, Training Loss: 0.6503868222236633, Validation Loss: 0.623038113117218
Epoch 10/50, Training Loss: 0.5316418766975403, Validation Loss: 0.46667447686195374
Epoch 20/50, Training Loss: 0.5034912288188934, Validation Loss: 0.4493418037891388
Epoch 30/50, Training Loss: 0.4874648451805115, Validation Loss: 0.4612681567668915
Epoch 40/50, Training Loss: 0.43948936462402344, Validation Loss: 0.49370817840099335


[I 2024-06-04 15:12:03,443] Trial 1 finished with value: 0.35036854445934296 and parameters: {'lr': 0.002509329138907062, 'dropout_prob': 0.269760735015655, 'hidden_channels': 134, 'n_layers': 5, 'kernel_size': 3, 'stride': 1, 'padding': 8, 'pooling_type': 'avg', 'use_batch_norm': True}. Best is trial 1 with value: 0.35036854445934296.


...

Epoch 50/50, Training Loss: 0.2767592340707779, Validation Loss: 0.14931069873273373
Epoch 10/50, Training Loss: 0.510171365737915, Validation Loss: 0.3399292081594467
Epoch 20/50, Training Loss: 0.41324399709701537, Validation Loss: 0.34380170702934265
Epoch 30/50, Training Loss: 0.42012616991996765, Validation Loss: 0.3240659534931183
Epoch 40/50, Training Loss: 0.33857416212558744, Validation Loss: 0.25059614330530167


[I 2024-06-04 15:32:57,369] Trial 98 finished with value: 0.29076844453811646 and parameters: {'lr': 0.00998291287825386, 'dropout_prob': 0.44565689940354536, 'hidden_channels': 171, 'n_layers': 4, 'kernel_size': 5, 'stride': 4, 'padding': 4, 'pooling_type': 'max', 'use_batch_norm': True}. Best is trial 24 with value: 0.11209205910563469.


Epoch 50/50, Training Loss: 0.3856714963912964, Validation Loss: 0.29076844453811646
Epoch 10/50, Training Loss: 0.560694795846939, Validation Loss: 0.4049019068479538
Epoch 20/50, Training Loss: 0.48152571320533755, Validation Loss: 0.33831748366355896
Epoch 30/50, Training Loss: 0.39571828842163087, Validation Loss: 0.23809681460261345
Epoch 40/50, Training Loss: 0.4637687087059021, Validation Loss: 0.33175236731767654


[I 2024-06-04 15:33:06,758] Trial 99 finished with value: 0.3481798395514488 and parameters: {'lr': 0.007645722744081645, 'dropout_prob': 0.4679224910840059, 'hidden_channels': 145, 'n_layers': 5, 'kernel_size': 5, 'stride': 3, 'padding': 7, 'pooling_type': 'max', 'use_batch_norm': False}. Best is trial 24 with value: 0.11209205910563469.


Epoch 50/50, Training Loss: 0.43611483573913573, Validation Loss: 0.3481798395514488


In [12]:
# Print the best hyperparameters and their associated final validation loss
print("Best hyperparameters:")
print(study.best_params)
print("Best final validation loss:")
print(study.best_value)

Best hyperparameters:
{'lr': 0.0027027086391409297, 'dropout_prob': 0.45718430173518243, 'hidden_channels': 88, 'n_layers': 4, 'kernel_size': 8, 'stride': 2, 'padding': 5, 'pooling_type': 'max', 'use_batch_norm': True}
Best final validation loss:
0.11209205910563469


In [14]:
# Specify the path where you want to save the JSON file
config_file_path = "../best_configs/force_press_in_best_configs_updated_classes.json"


In [16]:

# Load the saved configuration
with open(config_file_path, "r") as config_file:
    saved_configs = json.load(config_file)

# Compare the new best value with the saved one
if study.best_value < saved_configs["best_value"]:
    print("New best value found. Replacing saved configuration.")
    saved_configs["best_params"] = study.best_params
    saved_configs["best_value"] = study.best_value

    # Update the saved configuration file
    with open(config_file_path, "w") as config_file:
        json.dump(saved_configs, config_file)

In [15]:
# Save the best hyperparameters and validation loss to a JSON file
best_configs = {
    "best_params": study.best_params,
    "best_value": study.best_value,
}

with open(config_file_path, "w") as config_file:
    json.dump(best_configs, config_file)

# Thank You